In [1]:
!pip install -q "smolagents[litellm]" google-search-results markdownify beautifulsoup4 python-dotenv

### Open Deep Reseach Agent 재현해보기!

이 실습에서는 Hugging Face smolagents 예제인 open_deep_research의 구조를 그대로 따라가며, 여러 도구와 두 종류의 Agent(Worker, Manager)를 조합하여 정교한 리서치 Agent를 직접 구현합니다.

🎯 학습 목표
상태를 가지는(Stateful) 브라우저와 이를 제어하는 **도구 클래스(Tool Class)**를 직접 구현합니다.

ToolCallingAgent(Worker)와 CodeAgent(Manager)의 역할을 이해하고 Agent 팀을 구성합니다.

실제 프로젝트 수준의 Agent 아키텍처를 경험하고, 복잡한 문제를 해결하는 과정을 배웁니다.

리서치 Agent는 두 가지 핵심 API 키가 필요합니다.

OpenAI API Key: Agent의 두뇌 역할을 하는 LLM을 사용하기 위함입니다.

Serper API Key: 구글 검색 결과를 빠르고 안정적으로 가져오기 위함입니다. (serper.dev에서 무료 키를 발급, 하루 150회 사용 가능)

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if "OPENAI_API_KEY" not in os.environ or "SERPER_API_KEY" not in os.environ:
    print("⚠️ API 키가 설정되지 않았습니다. .env 파일을 확인해주세요!")
else:
    print("✅ API 키가 성공적으로 로드되었습니다.")

# Serper API 키를 SerpAPI 키로 인식하도록 환경 변수 이름을 변경해줍니다.
# (smolagents의 GoogleSearchTool이 내부적으로 이 변수명을 사용합니다.)
os.environ["SERPAPI_API_KEY"] = os.environ["SERPER_API_KEY"]

✅ API 키가 성공적으로 로드되었습니다.


### 단계 1: Agent의 핵심 도구 - SimpleTextBrowser 클래스 이해하기

단순한 검색 함수를 넘어, open_deep_research 예제는 **상태를 기억하는 SimpleTextBrowser**를 사용합니다. 마치 사람이 웹서핑 하듯, 현재 어떤 페이지에 있는지, 페이지의 내용은 무엇인지 등을 기억하는 똑똑한 브라우저입니다.


In [3]:
import requests
from markdownify import markdownify as md
from bs4 import BeautifulSoup

class SimpleTextBrowser:
    """간단한 텍스트 기반 웹 브라우저 클래스"""
    def __init__(self, start_page: str = "about:blank"):
        self.history = []
        self.page_content = ""
        self.page_title = ""
        self.visit_page(start_page)

    @property
    def address(self) -> str:
        """현재 페이지의 주소를 반환합니다."""
        return self.history[-1] if self.history else ""

    def visit_page(self, url: str) -> str:
        """주어진 URL을 방문하고 페이지 콘텐츠를 업데이트합니다."""
        print(f"👉 Browser navigating to: {url}")
        self.history.append(url)
        
        if url == "about:blank":
            self.page_content = "Blank page"
            self.page_title = "Blank"
            return self.page_content

        try:
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
            response = requests.get(url, headers=headers)
            response.raise_for_status() # 오류가 있으면 예외 발생
            
            # HTML을 Markdown으로 변환
            soup = BeautifulSoup(response.text, 'html.parser')
            self.page_title = soup.title.string if soup.title else "No Title"
            
            # body 태그 내용만 추출하여 더 깔끔하게 만듭니다.
            body = soup.find('body')
            if body:
                self.page_content = md(str(body))
            else:
                self.page_content = md(response.text)

        except requests.exceptions.RequestException as e:
            self.page_content = f"Error: Could not load page at {url}. Reason: {e}"
            self.page_title = "Error"
            
        return self.page_content

# 브라우저가 잘 작동하는지 테스트해봅시다.
browser_instance = SimpleTextBrowser()
content = browser_instance.visit_page("https://www.google.com")
print(f"\n--- 페이지 제목: {browser_instance.page_title} ---")
# 내용이 너무 기므로 앞 500자만 출력합니다.
print(content[:500])

👉 Browser navigating to: about:blank
👉 Browser navigating to: https://www.google.com

--- 페이지 제목: Google ---
[Google 정보](https://about.google/?fg=1&utm_source=google-KR&utm_medium=referral&utm_campaign=hp-header)[스토어](https://store.google.com/KR?utm_source=hp_header&utm_medium=google_ooo&utm_campaign=GS100042&hl=ko-KR)

[Gmail](https://mail.google.com/mail/&ogbl)

[이미지](https://www.google.com/imghp?hl=ko&ogbl)

[로그인](https://accounts.google.com/ServiceLogin?hl=ko&passive=true&continue=https://www.google.com/&ec=futura_exp_og_so_72776762_e)

![](/tia/tia.png)

- ![]()

  ![]()![]()

  ---

  더보기

  삭제
-


### 단계 2: Browser를 제어하는 '도구 클래스' 만들기
이제 SimpleTextBrowser 인스턴스를 제어할 Tool들을 만듭니다. smolagents의 Tool 클래스를 상속받아, Agent가 사용할 수 있는 표준화된 도구를 만듭니다.

In [ ]:
from smolagents import Tool, GoogleSearchTool

# 1. 페이지 방문 도구
class VisitTool(Tool):
    name = "visit_page"
    description = "주어진 URL의 웹페이지를 방문하여 텍스트 내용을 반환합니다."
    inputs = {"url": {"type": "string", "description": "방문할 웹페이지의 절대 주소"}}
    output_type = "string"

    def __init__(self, browser: SimpleTextBrowser):
        super().__init__()
        self.browser = browser

    def forward(self, url: str) -> str:
        return self.browser.visit_page(url)

# 2. 우리는 GoogleSearchTool은 smolagents에서 바로 가져와 사용!!
#    -> 이 도구는 SERPER_API_KEY(SERPAPI_API_KEY)를 사용하여 검색을 수행합니다.

# 생성한 도구들을 인스턴스화 합니다.
# 이 때, 1단계에서 만든 browser_instance를 VisitTool에 넘겨주는 것이 핵심!
browser_tools = [
    GoogleSearchTool(provider="serper"),
    VisitTool(browser_instance),
]

print("✅ 브라우저 제어 도구들이 준비되었습니다.")
print(f"사용 가능한 도구: {[tool.name for tool in browser_tools]}")

✅ 브라우저 제어 도구들이 준비되었습니다.
사용 가능한 도구: ['web_search', 'visit_page']


### 단계 3: Agent 팀 구성하기 (Worker & Manager)
open_deep_research의 핵심은 역할을 분담하는 Agent 팀입니다.

Worker (search_agent): ToolCallingAgent로, 우리가 만든 브라우저 도구들을 사용하여 정보 수집만 전문적으로 수행합니다.

Manager (manager_agent): CodeAgent로, 사용자의 질문을 받아 전체 리서치 계획을 세웁니다. Worker에게 작업을 지시하고, 받은 결과를 분석/가공하여 최종 보고서를 만듭니다. Python 코드를 실행할 수 있어 데이터 처리나 계산도 가능합니다.

In [6]:
from smolagents import ToolCallingAgent, CodeAgent, LiteLLMModel

# LLM 모델 정의
model = LiteLLMModel(model_id="gpt-4.1-mini")

# 1. Worker Agent (ToolCallingAgent) 생성
search_agent = ToolCallingAgent(
    model=model,
    tools=browser_tools, # 2단계에서 만든 도구 목록을 전달
    name="search_agent",
    description="""웹 검색 및 페이지 방문을 전문으로 하는 리서처입니다. 특정 정보가 필요할 때 이 Agent에게 명확한 지시를 내리세요.""",
    verbosity_level=2, # Agent의 작업 과정을 상세히 보려면 2로 설정
)

print("✅ Worker Agent (search_agent) 생성 완료!")

# 2. Manager Agent (CodeAgent) 생성
manager_agent = CodeAgent(
    model=model,
    tools=[],
    managed_agents=[search_agent], # 관리할 Worker Agent를 지정합니다.
    verbosity_level=2
)

print("✅ Manager Agent 생성 완료! 이제 리서치를 시작할 준비가 되었습니다.")

✅ Worker Agent (search_agent) 생성 완료!
✅ Manager Agent 생성 완료! 이제 리서치를 시작할 준비가 되었습니다.


### 단계 4: Deep Research Agent 실행 및 결과 확인
이제 모든 준비가 끝났습니다. Manager Agent에게 리서치 임무를 주고, Worker와 어떻게 협력하여 문제를 해결하는지 지켜봅시다.

In [7]:
# 리서치할 질문을 정의합니다.
question = "smol-agent와 LangChain의 주요 차이점은 무엇이고, 각각 어떤 경우에 사용하는 것이 더 적합한가요?"

print(f"🤔 질문: {question}")
print("\n🚀 Manager Agent가 리서치를 시작합니다...")
print("-" * 30)

# Manager Agent에게 질문을 전달하여 작업을 실행합니다.
final_report = manager_agent.run(question)

🤔 질문: smol-agent와 LangChain의 주요 차이점은 무엇이고, 각각 어떤 경우에 사용하는 것이 더 적합한가요?

🚀 Manager Agent가 리서치를 시작합니다...
------------------------------


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ smol-agent와 LangChain의 주요 차이점은 무엇이고, 각각 어떤 경우에 사용하는 것이 더 적합한가요?                  │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: The user wants to know the main differences between smol-agent and LangChain and the situations where each
is more suitable. This is a comparative question about two software/tools, likely related to AI or language models.
I should search for reliable information or documentation about both smol-agent and LangChain to understand their  
key features and use cases. I will ask the search_agent to gather information on the differences and suitable use  
cases for both smol-agent and LangChain.                                                                           
                                                                                                                   
<code>                                                                                                             
task_description = (                                                                                               
    "Please provide detailed information on the main differences between smol-agent and LangChain. "               
    "Additionally, explain in which scenarios or use cases each of these tools is more appropriate to use."        
)                                                                                                                  
                                                                                                                   
info = search_agent(task=task_description, additional_args={})                                                     
print(info)                                                                                                        
</code>                                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task_description = (                                                                                             
      "Please provide detailed information on the main differences between smol-agent and LangChain. "             
      "Additionally, explain in which scenarios or use cases each of these tools is more appropriate to use."      
  )                                                                                                                
                                                                                                                   
  info = search_agent(task=task_description, additional_args={})                                                   
  print(info)                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I have detailed information about the differences between smol-agent and LangChain and their respective   
use cases from the managed search agent. I will now summarize this information clearly, highlighting the main      
differences and the scenarios in which each one is more suitable, and provide it as the final answer.              
                                                                                                                   
<code>                                                                                                             
summary = (                                                                                                        
    "smol-agent and LangChain differ primarily in design philosophy and use cases:\n\n"                            
    "- LangChain is a modular framework designed for building LLM-driven applications with flexible chains, memory,
and tool integration. "                                                                                            
    "It is ideal for multi-step reasoning workflows, complex chatbot development, conversational memory, and       
orchestrating multiple APIs or services.\n"                                                                        
    "- smol-agent focuses on LLM-generated Python code execution, enabling transparent, code-centric, multi-step   
task solving, especially useful for advanced logic, calculations, "                                                
    "and data manipulation in sandboxed environments.\n\n"                                                         
    "Use Cases:\n"                                                                                                 
    "- Use LangChain when you need complex conversational agents, retrieval-augmented generation, multi-tool       
workflows, and scalable chatbot applications with memory.\n"                                                       
    "- Use smol-agent when your tasks require precise code execution, advanced math or data parsing, [38;2;230;237;243;48;2;13;17

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  summary = (                                                                                                      
      "smol-agent and LangChain differ primarily in design philosophy and use cases:\n\n"                          
      "- LangChain is a modular framework designed for building LLM-driven applications with flexible chains,      
  memory, and tool integration. "                                                                                  
      "It is ideal for multi-step reasoning workflows, complex chatbot development, conversational memory, and     
  orchestrating multiple APIs or services.\n"                                                                      
      "- smol-agent focuses on LLM-generated Python code execution, enabling transparent, code-centric,            
  multi-step task solving, especially useful for advanced logic, calculations, "                                   
      "and data manipulation in sandboxed environments.\n\n"                                                       
      "Use Cases:\n"                                                                                               
      "- Use LangChain when you need complex conversational agents, retrieval-augmented generation, multi-tool     
  workflows, and scalable chatbot applications with memory.\n"                                                     
      "- Use smol-agent when your tasks require precise code execution, advanced math or data parsing, or when     
  you prefer an open-source, code-based approach to reasoning.\n\n"                                                
      "In summary, LangChain excels at workflow orchestration and conversation-centric applications, while         
  smol-agent shines in computation-heavy, code-driven problem solving."                                            
  )                                                                                                                
                                                                                                                   
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: smol-agent and LangChain differ primarily in design philosophy and use cases:

- LangChain is a modular framework designed for building LLM-driven applications with flexible chains, memory, and 
tool integration. It is ideal for multi-step reasoning workflows, complex chatbot development, conversational 
memory, and orchestrating multiple APIs or services.
- smol-agent focuses on LLM-generated Python code execution, enabling transparent, code-centric, multi-step task 
solving, especially useful for advanced logic, calculations, and data manipulation in sandboxed environments.

Use Cases:
- Use LangChain when you need complex conversational agents, retrieval-augmented generation, multi-tool workflows, 
and scalable chatbot applications with memory.
- Use smol-agent when your tasks require precise code execution, advanced math or data parsing, or when you prefer 
an open-source, code-based approach to reasoning.

In summary, LangChain excels at workflow orchestration and conversation-centric applications, while smol-agent 
shines in computation-heavy, code-driven problem solving.

[Step 2: Duration 6.41 seconds| Input tokens: 5,109 | Output tokens: 460]

In [8]:
print("="*30)
print("✨ 최종 리서치 보고서 ✨")
print("="*30)
print(final_report)

✨ 최종 리서치 보고서 ✨
smol-agent and LangChain differ primarily in design philosophy and use cases:

- LangChain is a modular framework designed for building LLM-driven applications with flexible chains, memory, and tool integration. It is ideal for multi-step reasoning workflows, complex chatbot development, conversational memory, and orchestrating multiple APIs or services.
- smol-agent focuses on LLM-generated Python code execution, enabling transparent, code-centric, multi-step task solving, especially useful for advanced logic, calculations, and data manipulation in sandboxed environments.

Use Cases:
- Use LangChain when you need complex conversational agents, retrieval-augmented generation, multi-tool workflows, and scalable chatbot applications with memory.
- Use smol-agent when your tasks require precise code execution, advanced math or data parsing, or when you prefer an open-source, code-based approach to reasoning.

In summary, LangChain excels at workflow orchestration and conver

### Agent 팀의 '협업 과정' 엿보기

Manager와 Worker가 어떤 대화를 주고받았고, Worker가 어떤 도구를 어떻게 사용했는지 manager_agent.memory를 통해 상세하게 확인할 수 있습니다. 이것이 Agent의 작동 원리를 이해하는 가장 좋은 방법입니다.

In [ ]:
import json

# 1. Agent의 전체 작업 기록을 ChatMessage 객체 리스트로 가져옵니다.
agent_transcript_objects = manager_agent.write_memory_to_messages()

# 2. JSON으로 변환할 수 있도록 ChatMessage 객체 리스트를 dictionary 리스트로 변환합니다.
transcript_as_dicts = []
for msg in agent_transcript_objects:
    # 모든 메시지가 특정 속성을 가지고 있지는 않으므로, hasattr로 확인 후 추가합니다.
    message_dict = {
        'role': msg.role,
        'content': msg.content if hasattr(msg, 'content') and msg.content is not None else ''
    }
    
    # 도구 호출 관련 정보가 있는 경우에만 딕셔너리에 추가합니다.
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        message_dict['tool_calls'] = msg.tool_calls
        
    if hasattr(msg, 'tool_call_id') and msg.tool_call_id:
        message_dict['tool_call_id'] = msg.tool_call_id
        
    transcript_as_dicts.append(message_dict)


# 3. 이제 딕셔너리 리스트는 JSON으로 안전하게 변환할 수 있습니다.
print("="*30)
print("🧠 Agent 팀의 협업 과정 (Memory) 엿보기")
print("="*30)

print(json.dumps(transcript_as_dicts, indent=2, ensure_ascii=False))

🧠 Agent 팀의 협업 과정 (Memory) 엿보기
[
  {
    "role": "system",
    "content": [
      {
        "type": "text",
        "text": "You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', '<code>', and 'Observation:' sequences.\n\nAt each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the '<code>' sequence, you should write the code in simple Python. The code sequence must end with '</code>' sequence.\nDuring each intermediate step, you can use 'print()' to save whatever important information you will then need.\nThese print outputs will then appear in the 'Observation:' field, which will be available as inp

#### 위 코드 베이스를 바탕으로 새로운 것들을 붙이고, 나만의 Open Deep Research Agent를 만들어보세요